In [2]:
import json  
import os, sys, pathlib
import pickle

import pandas as pd
import matplotlib.pyplot as plt
import cv2

import importlib

WORK_DIR = pathlib.Path.cwd().parent


In [3]:
sys.path.append(str(WORK_DIR))

from src import ollama_manager as om
from src import main
from src.STAR_utils.visualization_tools import qa_visualization as qaviz


In [53]:
# please replace following data path to your local path
# all data can be download from our homepages (http://star.csail.mit.edu or https://bobbywu.com/STAR)

# root_dir = '/root/user/'
raw_video_dir = WORK_DIR / 'data/datasets/action-genome/Charades_v1_480/'
raw_frame_dir = WORK_DIR / 'data/datasets/action-genome/frames/'
annotation_dir = WORK_DIR / 'src/STAR_utils/annotations/STAR_classes/'
# pose_dir = root_dir + 'STAR/pose/'
# qa_train_dir = WORK_DIR /'data/datasets/STAR/STAR_annotations/STAR_train.json' 
qa_val_data = WORK_DIR / 'data/datasets/STAR/STAR_annotations/STAR_val.json' 
# qa_test_dir = BASE_DIR / 'data/datasets/STAR/STAR_annotations/STAR_test.json' 
save_video_dir = WORK_DIR / 'visualization_tmp'
fps = pickle.load(open(annotation_dir / 'video_fps','rb'))


In [54]:
star_data = []
with open(qa_val_data, 'r') as f:
    star_data = json.load(f)


In [ ]:

star_df = pd.DataFrame(star_data)
star_df = star_df.set_index('question_id')
star_df


,question,video_id,start,end,answer,question_program,choices,situations
question_id,,,,,,,,
Interaction_T1_13,Which object was tidied up by the person?,6H78U,11.1,19.6,The clothes.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'The closet/cabine...","{'000206': {'rel_pairs': [['o000', 'o027'], ['..."
Interaction_T1_14,Which object was tidied up by the person?,6H78U,15.6,22.7,The clothes.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'The blanket.', 'c...","{'000289': {'rel_pairs': [['o000', 'o027'], ['..."
Interaction_T1_31,Which object was thrown by the person?,RNLTR,7.3,13.5,The clothes.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'The pillow.', 'ch...","{'000132': {'rel_pairs': [['o000', 'o019'], ['..."
Interaction_T1_32,Which object was put down by the person?,RNLTR,11.6,16.4,The shoe.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'The food.', 'choi...","{'000202': {'rel_pairs': [['o000', 'o019'], ['..."
Interaction_T1_40,Which object was tidied up by the person?,VNQTH,2.7,9.2,The closet/cabinet.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'The broom.', 'cho...","{'000110': {'rel_pairs': [['o000', 'o014'], ['..."
...,...,...,...,...,...,...,...,...
Feasibility_T6_1453,What is the person able to do after taking the...,L9ANI,7.3,12.9,Wash the table.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'Wash the table.',...","{'000227': {'rel_pairs': [['o000', 'o006'], ['..."
Feasibility_T6_1454,What is the person able to do after walking th...,L9ANI,0.0,4.1,Take the towel.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'Take the towel.',...","{'000033': {'rel_pairs': [['o000', 'o006']], '..."
Feasibility_T6_1455,What is the person able to do after walking th...,L9ANI,0.0,4.1,Wash the table.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'Throw the bag.', ...","{'000033': {'rel_pairs': [['o000', 'o006']], '..."


In [8]:
gt_stsg = []
with open(WORK_DIR / "data/datasets/STAR_verbalized_stsg_val.json") as f:
    gt_stsg = json.load(f)
    

gt_stsg_df = pd.DataFrame(gt_stsg)
gt_stsg_df.iloc[0]


question_id                                    Interaction_T1_13
video_id                                                   6H78U
start                                                       11.1
end                                                         19.6
stsg           Frame 000198\n\tperson --- on_the_side_of --- ...
Name: 0, dtype: object

In [7]:
gen_stsg = []
with open(WORK_DIR / "outputs/gen_stsg_gemma3:12b-it-qat_20250522_14:30_99.jsonl", 'r') as f:
    gen_stsg = [json.loads(line) for line in f.readlines()]
    
gen_stsg_df = pd.DataFrame(gen_stsg)
gen_stsg_df.iloc[0]


video_id                                                    6H78U
start                                                        15.6
end                                                          22.7
chat_history    [{'role': 'user', 'content': 'Look carefully a...
stsg            \nFrame 0:\n\nyoung_man ---- standing_on ---- ...
Name: 0, dtype: object

In [10]:
gt_stsg_df[['stsg']].value_counts().shape


(2883,)

In [48]:
gt_stsg_df[['video_id', 'start', 'end']].value_counts().shape


(3373,)

In [60]:
star_df.iloc[0]['situations'].keys()


dict_keys(['000206', '000212', '000217', '000221', '000218', '000236', '000223', '000243', '000242', '000240', '000247', '000257', '000245', '000270', '000267', '000269', '000272', '000274', '000275', '000289', '000286', '000294', '000301', '000303', '000313', '000308', '000318', '000316', '000314', '000320', '000322', '000332', '000331', '000336', '000337', '000339', '000198', '000205', '000202'])

In [62]:
result = (
    star_df.groupby(['video_id', 'start', 'end'])['situations']
    .agg(lambda x: len(set(frozenset(d.keys()) for d in x)) == 1)
)
result


video_id  start  end  
013SD     0.0    4.30      True
01KML     18.1   26.80     True
01ZWG     4.9    11.00     True
                 13.50     True
          7.2    13.50     True
                          ...  
ZZ89F     0.0    9.60     False
                 15.35     True
          7.4    13.50     True
          13.6   20.60     True
                 26.50     True
Name: situations, Length: 3373, dtype: bool

In [82]:
star_df.loc[(star_df['video_id'] == 'ZZ89F') & (star_df['start'] == 0.0) & (star_df['end'] == 9.6)]['situations'].apply(lambda x: x.keys()).to_list()


[dict_keys(['000072', '000073', '000083', '000078', '000085', '000097', '000094', '000105', '000116', '000108', '000142', '000129', '000120', '000166', '000179', '000154', '000027', '000010', '000061', '000044']),
 dict_keys(['000072', '000073', '000083', '000078', '000085', '000097', '000094', '000105', '000116', '000108', '000142', '000129', '000120', '000166', '000179', '000154', '000027', '000010', '000061', '000044']),
 dict_keys(['000072', '000073', '000083', '000078', '000085', '000097', '000094', '000105', '000116', '000108', '000142', '000129', '000120', '000166', '000179', '000154']),
 dict_keys(['000072', '000073', '000083', '000078', '000085', '000097', '000094', '000105', '000116', '000108', '000142', '000129', '000120', '000166', '000179', '000154'])]

In [49]:
gt_stsg_df.groupby(['video_id', 'start', 'end'])['stsg'].agg(lambda x: x.nunique() == 1)


video_id  start  end  
013SD     0.0    4.30      True
01KML     18.1   26.80     True
01ZWG     4.9    11.00     True
                 13.50     True
          7.2    13.50     True
                          ...  
ZZ89F     0.0    9.60     False
                 15.35     True
          7.4    13.50     True
          13.6   20.60     True
                 26.50     True
Name: stsg, Length: 3373, dtype: bool

In [77]:
gt_stsg_df.loc[(gt_stsg_df['video_id'] == 'ZZ89F') & (gt_stsg_df['start'] == 0.0) & (gt_stsg_df['end'] == 9.6)]['stsg']


6716    Frame 000010\n\ttowel --- near --- shoe\nFrame...
6729    Frame 000010\n\ttowel --- near --- shoe\nFrame...
6913    Frame 000072\n\tperson --- in_front_of --- ref...
7067    Frame 000072\n\tperson --- in_front_of --- ref...
Name: stsg, dtype: object

In [94]:
gen_stsg_df[['video_id', 'start', 'end']]


,video_id,start,end
0,6H78U,15.6,22.7
1,RNLTR,11.6,16.4
2,VNQTH,2.7,9.2
3,Y79PC,16.9,22.6
4,4GLAP,15.7,21.4
...,...,...,...
1089,YCGJS,8.0,14.4
1090,86GSE,10.1,20.0
1091,9J166,18.6,23.0
1092,ZQAMW,19.4,24.2


In [92]:
gen_stsg_df.groupby(['video_id', 'start', 'end'])[['stsg']].agg(lambda x: x.nunique() == 1).sum()


stsg    841
dtype: int64

In [116]:
idx = 'Feasibility_T6_978'
video_id = star_df.loc[idx]['video_id']
start = star_df.loc[idx]['start']
end = star_df.loc[idx]['end']


In [117]:
gt_stsg = gt_stsg_df.loc[gt_stsg_df['question_id'] == idx]['stsg'].values[0]


In [118]:
gen_stsg = gen_stsg_df[(gen_stsg_df['video_id'] == video_id) & (gen_stsg_df['start'] == start) & (gen_stsg_df['end'] == end)]['stsg'].values[0]


In [121]:
print(gt_stsg)


Frame 000106
	person --- holding --- clothes
	person --- in_front_of --- clothes
	person --- sitting_on --- sofa/couch
	person --- behind --- sofa/couch
Frame 000120
	person --- holding --- clothes
	person --- in_front_of --- clothes
	person --- sitting_on --- sofa/couch
	person --- beneath --- sofa/couch
	person --- behind --- sofa/couch
	person --- holding --- blanket
	person --- in_front_of --- blanket
Frame 000137
	person --- holding --- clothes
	person --- in_front_of --- clothes
	person --- sitting_on --- sofa/couch
	person --- behind --- sofa/couch
	person --- beneath --- sofa/couch
Frame 000167
	person --- holding --- clothes
	person --- in_front_of --- clothes
	person --- sitting_on --- sofa/couch
	person --- behind --- sofa/couch
	person --- beneath --- sofa/couch
	person --- holding --- blanket
	person --- in_front_of --- blanket
Frame 000168
	person --- holding --- clothes
	person --- in_front_of --- clothes
	person --- sitting_on --- sofa/couch
	person --- leaning_on --- s

In [120]:
print(gen_stsg)



Frame 0:


woman ---- holding ---- white_plate
white_plate ---- contains ---- food_on_plate
woman ---- sitting_on ---- sofa
sofa ---- has_cushions ---- sofa_cushions
sofa_cushions ---- patterned ---- patterned_design
sofa ---- in_front_of ---- wall
wall ---- behind ---- sofa
wall ---- has_curtains ---- curtains
wall ---- has_picture_frame ---- picture_frame
picture_frame ---- on_top_of ---- sofa
light_fixture ---- above ---- woman
light_fixture ---- attached_to ---- ceiling
cabinet ---- to_right_of ---- woman
cabinet ---- displays ---- decorative_items
floor ---- below ---- woman
floor ---- below ---- sofa

Frame 1:

woman ---- sitting_on ---- sofa
sofa ---- has_cushion ---- cushion
cushion ---- patterned_with ---- floral_pattern
woman ---- holding ---- plate
woman ---- holding ---- fork
plate ---- in_front_of ---- woman
fork ---- in_front_of ---- woman
sofa ---- positioned_in_front_of ---- cabinet
cabinet ---- located_behind ---- sofa
cabinet ---- has_shelf ---- shelf
shelf ---- disp